# Food Magazine - Eat Safe, Love

## Part 1: Notebook Set Up

In [61]:
from pymongo import MongoClient
import pandas as pd
from pprint import pprint

# Define a dictionary structure for header row styling
df_header_row = {
    'selector': 'th.col_heading',
    'props': 'background-color: bisque;'
}

In [62]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [63]:
# assign the uk_food database to a variable name
db = mongo['uk_food']

In [64]:
# review the collections in our database
print(db.list_collection_names())

['establishments']


In [65]:
# assign the collection to a variable
establishments = db['establishments']

## Part 3: Exploratory Analysis
Unless otherwise stated, for each question: 
* Use `count_documents` to display the number of documents contained in the result.
* Display the first document in the results using `pprint`.
* Convert the result to a Pandas DataFrame, print the number of rows in the DataFrame, and display the first 10 rows.

### 1. Which establishments have a hygiene score equal to 20?

In [77]:
# Find the establishments with a hygiene score of 20
query = {"scores.Hygiene": 20}

# Use count_documents to display the number of documents in the result
documents_count_hygiene_score = establishments.count_documents(query)

# Display the first document in the results using pprint
results = establishments.find(query)
print("Number of documents with a Hygiene score of 20:", documents_count_hygiene_score)
print("")
print("The First Document in the results:")
print("-"*50)
pprint(results[0])



Number of documents with a Hygiene score of 20: 41

The First Document in the results:
--------------------------------------------------
{'AddressLine1': '5-6 Southfields Road',
 'AddressLine2': 'Eastbourne',
 'AddressLine3': 'East Sussex',
 'AddressLine4': '',
 'BusinessName': 'The Chase Rest Home',
 'BusinessType': 'Caring Premises',
 'BusinessTypeID': 5,
 'ChangesByServerID': 0,
 'Distance': 4613.888288172291,
 'FHRSID': 110681,
 'LocalAuthorityBusinessID': '4029',
 'LocalAuthorityCode': '102',
 'LocalAuthorityEmailAddress': 'Customerfirst@eastbourne.gov.uk',
 'LocalAuthorityName': 'Eastbourne',
 'LocalAuthorityWebSite': 'http://www.eastbourne.gov.uk/foodratings',
 'NewRatingPending': False,
 'Phone': '',
 'PostCode': 'BN21 1BU',
 'RatingDate': '2021-09-23T00:00:00',
 'RatingKey': 'fhrs_0_en-gb',
 'RatingValue': 0,
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('65fb9f10abd55ac823bd1a23'),
 'geocode': {'latitude': Decimal128('50.769705'),
             'longitude': Dec

In [78]:
# Convert the result to a Pandas DataFrame
result_df = pd.DataFrame(results)

# Display the number of rows in the DataFrame
print("Rows in DataFrame: ", len(result_df))

# Display the first 10 rows of the DataFrame
result_df.head(10).style.hide(axis="index").set_table_styles([df_header_row])


Rows in DataFrame:  41


_id,FHRSID,ChangesByServerID,LocalAuthorityBusinessID,BusinessName,BusinessType,BusinessTypeID,AddressLine1,AddressLine2,AddressLine3,AddressLine4,PostCode,Phone,RatingValue,RatingKey,RatingDate,LocalAuthorityCode,LocalAuthorityName,LocalAuthorityWebSite,LocalAuthorityEmailAddress,scores,SchemeType,geocode,RightToReply,Distance,NewRatingPending,meta,links
65fb9f10abd55ac823bd1a23,110681,0,4029,The Chase Rest Home,Caring Premises,5,5-6 Southfields Road,Eastbourne,East Sussex,,BN21 1BU,,0,fhrs_0_en-gb,2021-09-23T00:00:00,102,Eastbourne,http://www.eastbourne.gov.uk/foodratings,Customerfirst@eastbourne.gov.uk,"{'Hygiene': 20, 'Structural': 20, 'ConfidenceInManagement': 20}",FHRS,"{'longitude': Decimal128('0.27694'), 'latitude': Decimal128('50.769705')}",,4613.888288,False,"{'dataSource': None, 'extractDate': '0001-01-01T00:00:00', 'itemCount': 0, 'returncode': None, 'totalCount': 0, 'totalPages': 0, 'pageSize': 0, 'pageNumber': 0}","[{'rel': 'self', 'href': 'https://api.ratings.food.gov.uk/establishments/110681'}]"
65fb9f10abd55ac823bd1da2,612039,0,1970/FOOD,Brenalwood,Caring Premises,5,Hall Lane,Walton-on-the-Naze,Essex,,CO14 8HN,,0,fhrs_0_en-gb,2022-06-22T00:00:00,120,Tendring,http://www.tendringdc.gov.uk/,fhsadmin@tendringdc.gov.uk,"{'Hygiene': 20, 'Structural': 15, 'ConfidenceInManagement': 30}",FHRS,"{'longitude': Decimal128('1.278721'), 'latitude': Decimal128('51.857536')}",,4617.965824,False,"{'dataSource': None, 'extractDate': '0001-01-01T00:00:00', 'itemCount': 0, 'returncode': None, 'totalCount': 0, 'totalPages': 0, 'pageSize': 0, 'pageNumber': 0}","[{'rel': 'self', 'href': 'https://api.ratings.food.gov.uk/establishments/612039'}]"
65fb9f10abd55ac823bd20b4,730933,0,1698/FOOD,Melrose Hotel,Hotel/bed & breakfast/guest house,7842,53 Marine Parade East,Clacton On Sea,Essex,,CO15 6AD,,0,fhrs_0_en-gb,2021-08-13T00:00:00,120,Tendring,http://www.tendringdc.gov.uk/,fhsadmin@tendringdc.gov.uk,"{'Hygiene': 20, 'Structural': 20, 'ConfidenceInManagement': 20}",FHRS,"{'longitude': Decimal128('1.15927'), 'latitude': Decimal128('51.789429')}",,4619.656144,False,"{'dataSource': None, 'extractDate': '0001-01-01T00:00:00', 'itemCount': 0, 'returncode': None, 'totalCount': 0, 'totalPages': 0, 'pageSize': 0, 'pageNumber': 0}","[{'rel': 'self', 'href': 'https://api.ratings.food.gov.uk/establishments/730933'}]"
65fb9f10abd55ac823bd22a1,172735,0,PI/000023858,Seaford Pizza,Takeaway/sandwich shop,7844,4 High Street,Seaford,East Sussex,,BN25 1PG,,1,fhrs_1_en-gb,2021-12-23T00:00:00,105,Lewes,http://www.lewes-eastbourne.gov.uk/,ehealth.ldc@lewes-eastbourne.gov.uk,"{'Hygiene': 20, 'Structural': 10, 'ConfidenceInManagement': 20}",FHRS,"{'longitude': Decimal128('0.10202'), 'latitude': Decimal128('50.770885')}",,4620.421725,False,"{'dataSource': None, 'extractDate': '0001-01-01T00:00:00', 'itemCount': 0, 'returncode': None, 'totalCount': 0, 'totalPages': 0, 'pageSize': 0, 'pageNumber': 0}","[{'rel': 'self', 'href': 'https://api.ratings.food.gov.uk/establishments/172735'}]"
65fb9f10abd55ac823bd22b3,172953,0,PI/000024532,Golden Palace,Restaurant/Cafe/Canteen,1,5 South Street,Seaford,East Sussex,,BN25 1HP,,1,fhrs_1_en-gb,2021-10-07T00:00:00,105,Lewes,http://www.lewes-eastbourne.gov.uk/,ehealth.ldc@lewes-eastbourne.gov.uk,"{'Hygiene': 20, 'Structural': 10, 'ConfidenceInManagement': 20}",FHRS,"{'longitude': Decimal128('0.101446'), 'latitude': Decimal128('50.770724')}",,4620.437179,False,"{'dataSource': None, 'extractDate': '0001-01-01T00:00:00', 'itemCount': 0, 'returncode': None, 'totalCount': 0, 'totalPages': 0, 'pageSize': 0, 'pageNumber': 0}","[{'rel': 'self', 'href': 'https://api.ratings.food.gov.uk/establishments/172953'}]"
65fb9f10abd55ac823bd2c4c,512854,0,12/00816/BUTH,Ashby's Butchers,Retailers - other,4613,777 Southchurch Road,Southend-On-Sea,Essex,,SS1 2PP,,0,fhrs_0_en-gb,2022-07-21T00:00:00,893,Southend-On-Sea,http://www.southend.gov.uk,EnvironmentalHealth@southend.gov.uk,"{'Hygiene': 20, 'Structural': 20, 'ConfidenceInManagement': 20}",FHRS,"{'l

### 2. Which establishments in London have a `RatingValue` greater than or equal to 4?

In [68]:
# Find the establishments with London as the Local Authority and has a RatingValue greater than or equal to 4.
query = {"LocalAuthorityName": {'$regex': "London"}, "RatingValue":{'$gte': 4}}

# Use count_documents to display the number of documents in the result
documents_count_London_Rating_Value = establishments.count_documents(query)

# Display the first document in the results using pprint
results_ratings_value = establishments.find(query)
print("Number of documents for establisments in 'London' have a 'RatingValue' greater than or equal to 4:", documents_count_London_Rating_Value) 
print("")
print("The First Document in the results:")
print("-"*50)
pprint(results_ratings_value[0])


Number of documents for establisments in 'London' have a 'RatingValue' greater than or equal to 4: 33

The First Document in the results:
--------------------------------------------------
{'AddressLine1': 'Oak Apple Farm Building 103 Sheernes Docks',
 'AddressLine2': 'Sheppy Kent',
 'AddressLine3': '',
 'AddressLine4': '',
 'BusinessName': "Charlie's",
 'BusinessType': 'Other catering premises',
 'BusinessTypeID': 7841,
 'ChangesByServerID': 0,
 'Distance': 4627.439467780196,
 'FHRSID': 621707,
 'LocalAuthorityBusinessID': 'PI/000025307',
 'LocalAuthorityCode': '508',
 'LocalAuthorityEmailAddress': 'publicprotection@cityoflondon.gov.uk',
 'LocalAuthorityName': 'City of London Corporation',
 'LocalAuthorityWebSite': 'http://www.cityoflondon.gov.uk/Corporation/homepage.htm',
 'NewRatingPending': False,
 'Phone': '',
 'PostCode': 'ME12',
 'RatingDate': '2021-10-18T00:00:00',
 'RatingKey': 'fhrs_4_en-gb',
 'RatingValue': 4,
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('65f

In [69]:
# Convert the result to a Pandas DataFrame
london_rating_df = pd.DataFrame(results_ratings_value)

# Display the number of rows in the DataFrame
print("Rows in DataFrame: ", len(london_rating_df))

# Display the first 10 rows of the DataFrame
london_rating_df.head(10).style.hide(axis="index").set_table_styles([df_header_row])

Rows in DataFrame:  33


_id,FHRSID,ChangesByServerID,LocalAuthorityBusinessID,BusinessName,BusinessType,BusinessTypeID,AddressLine1,AddressLine2,AddressLine3,AddressLine4,PostCode,Phone,RatingValue,RatingKey,RatingDate,LocalAuthorityCode,LocalAuthorityName,LocalAuthorityWebSite,LocalAuthorityEmailAddress,scores,SchemeType,geocode,RightToReply,Distance,NewRatingPending,meta,links
65fb9f10abd55ac823bd343c,621707,0,PI/000025307,Charlie's,Other catering premises,7841,Oak Apple Farm Building 103 Sheernes Docks,Sheppy Kent,,,ME12,,4,fhrs_4_en-gb,2021-10-18T00:00:00,508,City of London Corporation,http://www.cityoflondon.gov.uk/Corporation/homepage.htm,publicprotection@cityoflondon.gov.uk,"{'Hygiene': 5, 'Structural': 10, 'ConfidenceInManagement': 5}",FHRS,"{'longitude': Decimal128('0.508551'), 'latitude': Decimal128('51.369321')}",,4627.439468,False,"{'dataSource': None, 'extractDate': '0001-01-01T00:00:00', 'itemCount': 0, 'returncode': None, 'totalCount': 0, 'totalPages': 0, 'pageSize': 0, 'pageNumber': 0}","[{'rel': 'self', 'href': 'https://api.ratings.food.gov.uk/establishments/621707'}]"
65fb9f10abd55ac823bd376a,1130836,0,PI/000034075,Mv City Cruises Erasmus,Other catering premises,7841,Cherry Garden Pier,Cherry Garden Street Rotherhithe,Charlton,,TN4 8HR,,5,fhrs_5_en-gb,2021-05-14T00:00:00,508,City of London Corporation,http://www.cityoflondon.gov.uk/Corporation/homepage.htm,publicprotection@cityoflondon.gov.uk,"{'Hygiene': 0, 'Structural': 5, 'ConfidenceInManagement': 0}",FHRS,"{'longitude': Decimal128('0.249255999922752'), 'latitude': Decimal128('51.1236228942871')}",,4627.873179,False,"{'dataSource': None, 'extractDate': '0001-01-01T00:00:00', 'itemCount': 0, 'returncode': None, 'totalCount': 0, 'totalPages': 0, 'pageSize': 0, 'pageNumber': 0}","[{'rel': 'self', 'href': 'http://api.ratings.food.gov.uk/establishments/1130836'}]"
65fb9f10abd55ac823bd42b0,293783,0,PI/000002614,Benfleet Motor Yacht Club,Other catering premises,7841,Ferry Road,South Benfleet Essex,,,SS7 1NF,,4,fhrs_4_en-gb,2018-11-02T00:00:00,508,City of London Corporation,http://www.cityoflondon.gov.uk/Corporation/homepage.htm,publicprotection@cityoflondon.gov.uk,"{'Hygiene': 0, 'Structural': 0, 'ConfidenceInManagement': 10}",FHRS,"{'longitude': Decimal128('0.561954'), 'latitude': Decimal128('51.543831')}",,4631.968418,False,"{'dataSource': None, 'extractDate': '0001-01-01T00:00:00', 'itemCount': 0, 'returncode': None, 'totalCount': 0, 'totalPages': 0, 'pageSize': 0, 'pageNumber': 0}","[{'rel': 'self', 'href': 'http://api.ratings.food.gov.uk/establishments/293783'}]"
65fb9f11abd55ac823bd50b0,1315095,0,PI/000036464,Coombs Catering t/a The Lock and Key,Restaurant/Cafe/Canteen,1,Leslie Ford House,Tilbury,Charlton,,RM18 7EH,,5,fhrs_5_en-gb,2020-12-04T00:00:00,508,City of London Corporation,http://www.cityoflondon.gov.uk/Corporation/homepage.htm,publicprotection@cityoflondon.gov.uk,"{'Hygiene': 0, 'Structural': 5, 'ConfidenceInManagement': 0}",FHRS,"{'longitude': Decimal128('0.346336990594864'), 'latitude': Decimal128('51.4640655517578')}",,4636.846754,False,"{'dataSource': None, 'extractDate': '0001-01-01T00:00:00', 'itemCount': 0, 'returncode': None, 'totalCount': 0, 'totalPages': 0, 'pageSize': 0, 'pageNumber': 0}","[{'rel': 'self', 'href': 'http://api.ratings.food.gov.uk/establishments/1315095'}]"
65fb9f11abd55ac823bd50b1,294474,0,PI/000014647,Tilbury Seafarers Centre,Restaurant/Cafe/Canteen,1,Tenants Row Tilbury Docks,Tilbury Essex,,,RM18 7EH,,5,fhrs_5_en-gb,2018-11-02T00:00:00,508,City of London Corporation,http://www.cityoflondon.gov.uk/Corporation/homepage.htm,publicprotection@cityoflondon.gov.uk,"{'Hygiene': 0, 'Structural': 0, 'ConfidenceInManagement': 0}",FHRS,"{'longitude': Decimal128('0.346325'), 'latitude': Decimal128('51.464078')}",,4636.847650,False,"{'dataSource': None, 'extractDate': '0001-01-01T00:00:00', 'itemCount': 0, 'returncode': None, 'totalCount': 0, 'totalPages': 0, 'pageSize': 0, 'pageNumber': 0}","[{'rel': 'self', 'href': 'http://api.ratings.food.gov.uk/establis

### 3. What are the top 5 establishments with a `RatingValue` rating value of 5, sorted by lowest hygiene score, nearest to the new restaurant added, "Penang Flavours"?

In [70]:
# Search within 0.01 degree on either side of the latitude and longitude.
# Rating value must equal 5
# Sort by hygiene score

# Assign the latitude and longitude values of the newly added restaurant 'Penang Flavours'
degree_search = 0.01
latitude = 51.49014200
longitude = 0.08384000

query = {
         'RatingValue' : 5,
         'geocode.latitude' : {'$gte' : latitude - degree_search, '$lte' : latitude + degree_search},
         'geocode.longitude': {'$gte' : longitude - degree_search, '$lte' : longitude + degree_search}
        }
sort = [("scores.Hygiene",1)]
limit = 5

# Print the results
results_top_establishments = list(establishments.find(query).sort(sort).limit(limit))

print('Number of establishments: ', len(results_top_establishments))
pprint(results_top_establishments)

Number of establishments:  5
[{'AddressLine1': 'Restaurant And Premises 107A Plumstead High Street',
  'AddressLine2': '',
  'AddressLine3': 'Plumstead',
  'AddressLine4': 'Greenwich',
  'BusinessName': 'Howe and Co Fish and Chips - Van 17',
  'BusinessType': 'Mobile caterer',
  'BusinessTypeID': 7846,
  'ChangesByServerID': 0,
  'Distance': 4646.95593107927,
  'FHRSID': 1380578,
  'LocalAuthorityBusinessID': '14425',
  'LocalAuthorityCode': '511',
  'LocalAuthorityEmailAddress': 'health@royalgreenwich.gov.uk',
  'LocalAuthorityName': 'Greenwich',
  'LocalAuthorityWebSite': 'http://www.royalgreenwich.gov.uk',
  'NewRatingPending': False,
  'Phone': '',
  'PostCode': 'SE18 1SE',
  'RatingDate': '2021-11-11T00:00:00',
  'RatingKey': 'fhrs_5_en-gb',
  'RatingValue': 5,
  'RightToReply': '',
  'SchemeType': 'FHRS',
  '_id': ObjectId('65fb9f11abd55ac823bd7264'),
  'geocode': {'latitude': Decimal128('51.4875335693359'),
              'longitude': Decimal128('0.0925370007753372')},
  'links':

In [79]:
# Convert result to Pandas DataFrame
restaurants_near_penang =pd.json_normalize(results_top_establishments)

# Display the Pandas Dataframe
restaurants_near_penang.style.hide(axis="index").set_table_styles([df_header_row])

_id,FHRSID,ChangesByServerID,LocalAuthorityBusinessID,BusinessName,BusinessType,BusinessTypeID,AddressLine1,AddressLine2,AddressLine3,AddressLine4,PostCode,Phone,RatingValue,RatingKey,RatingDate,LocalAuthorityCode,LocalAuthorityName,LocalAuthorityWebSite,LocalAuthorityEmailAddress,SchemeType,RightToReply,Distance,NewRatingPending,links,scores.Hygiene,scores.Structural,scores.ConfidenceInManagement,geocode.longitude,geocode.latitude,meta.dataSource,meta.extractDate,meta.itemCount,meta.returncode,meta.totalCount,meta.totalPages,meta.pageSize,meta.pageNumber
65fb9f11abd55ac823bd7264,1380578,0,14425,Howe and Co Fish and Chips - Van 17,Mobile caterer,7846,Restaurant And Premises 107A Plumstead High Street,,Plumstead,Greenwich,SE18 1SE,,5,fhrs_5_en-gb,2021-11-11T00:00:00,511,Greenwich,http://www.royalgreenwich.gov.uk,health@royalgreenwich.gov.uk,FHRS,,4646.955931,False,"[{'rel': 'self', 'href': 'http://api.ratings.food.gov.uk/establishments/1380578'}]",0,0,0,0.0925370007753372,51.4875335693359,None,0001-01-01T00:00:00,0,None,0,0,0,0
65fb9f11abd55ac823bd729f,694478,0,PI/000086506,Atlantic Fish Bar,Takeaway/sandwich shop,7844,35 Lakedale Road,,Plumstead,Greenwich,SE18 1PR,,5,fhrs_5_en-gb,2021-06-16T00:00:00,511,Greenwich,http://www.royalgreenwich.gov.uk,health@royalgreenwich.gov.uk,FHRS,,4646.974612,False,"[{'rel': 'self', 'href': 'http://api.ratings.food.gov.uk/establishments/694478'}]",0,0,5,0.0912164,51.4867296,None,0001-01-01T00:00:00,0,None,0,0,0,0
65fb9f11abd55ac823bd7299,695241,0,PI/000179088,Plumstead Manor Nursery,Caring Premises,5,Plumstead Manor School Old Mill Road,,Plumstead,Greenwich,SE18 1QG,,5,fhrs_5_en-gb,2021-06-16T00:00:00,511,Greenwich,http://www.royalgreenwich.gov.uk,health@royalgreenwich.gov.uk,FHRS,,4646.974010,False,"[{'rel': 'self', 'href': 'http://api.ratings.food.gov.uk/establishments/695241'}]",0,0,5,0.0859939977526665,51.481517791748,None,0001-01-01T00:00:00,0,None,0,0,0,0
65fb9f11abd55ac823bd7253,695223,0,PI/000178842,Iceland,Retailers - supermarkets/hypermarkets,7840,144 - 146 Plumstead High Street,,Plumstead,Greenwich,SE18 1JQ,,5,fhrs_5_en-gb,2019-11-13T00:00:00,511,Greenwich,http://www.royalgreenwich.gov.uk,health@royalgreenwich.gov.uk,FHRS,,4646.946071,False,"[{'rel': 'self', 'href': 'http://api.ratings.food.gov.uk/establishments/695223'}]",0,5,5,0.0924199968576431,51.4871482849121,None,0001-01-01T00:00:00,0,None,0,0,0,0
65fb9f11abd55ac823bd7282,694609,0,PI/000116619,Volunteer,Pub/bar/nightclub,7843,130 - 132 Plumstead High Street,,Plumstead,Greenwich,SE18 1JQ,,5,fhrs_5_en-gb,2019-08-05T00:00:00,511,Greenwich,http://www.royalgreenwich.gov.uk,health@royalgreenwich.gov.uk,FHRS,,4646.965635,False,"[{'rel': 'self', 'href': 'http://api.ratings.food.gov.uk/establishments/694609'}]",0,0,0,0.09208,51.4873437,None,0001-01-01T00:00:00,0,None,0,0,0,0


### 4. How many establishments in each Local Authority area have a hygiene score of 0?

In [80]:
# Create a pipeline that:
# 1. Matches establishments with a hygiene score of 0
# 2. Groups the matches by Local Authority
# 3. Sorts the matches from highest to lowest

# Pipeline
match_query = {'$match': {"scores.Hygiene": {'$eq':0}}}

group_query = {'$group': {"_id" : "$LocalAuthorityName", "count": {'$sum':1}}}

sort_query = {'$sort':{"count":-1}}

pipeline = [match_query, group_query, sort_query]

final_results = list(establishments.aggregate(pipeline))

# Print the number of documents in the result
print(f"There are {len(final_results)} documents in the result.")

# Print the first 10 results
print("")
print("The first 10 documents are:")
print("-"*50)
pprint(final_results[0:10])

There are 55 documents in the result.

The first 10 documents are:
--------------------------------------------------
[{'_id': 'Thanet', 'count': 1130},
 {'_id': 'Greenwich', 'count': 882},
 {'_id': 'Maidstone', 'count': 713},
 {'_id': 'Newham', 'count': 711},
 {'_id': 'Swale', 'count': 686},
 {'_id': 'Chelmsford', 'count': 680},
 {'_id': 'Medway', 'count': 672},
 {'_id': 'Bexley', 'count': 607},
 {'_id': 'Southend-On-Sea', 'count': 586},
 {'_id': 'Tendring', 'count': 542}]


In [83]:
# Convert the result to a Pandas DataFrame
final_results_df =pd.json_normalize(final_results)

# Display the number of rows in the DataFrame
print(f'The number of rows in the DataFrame is {len(final_results_df)}')

# Display the first 10 rows of the DataFrame
final_results_df.head(5).style.hide(axis="index").set_table_styles([df_header_row])

The number of rows in the DataFrame is 55


_id,count
Thanet,1130
Greenwich,882
Maidstone,713
Newham,711
Swale,686
